In [1]:
import os
import sys
sys.path.append("..")

import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision

#from data import loader_train, loader_val, loader_test
from train import train_model, test_model
from models import getModels
from utils import show_img
from trainers import Regression, Classification, BinaryClassification
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from ResNet18 import ResNet, BasicBlock

In [3]:
from medmnist import PneumoniaMNIST, BreastMNIST, RetinaMNIST, DermaMNIST, ChestMNIST, PathMNIST

from torch.utils.data import Subset

batch_size = 128
transfomrms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
test_dataset = DermaMNIST(split="test",transform= transfomrms, download=True,size=224)
train_dataset = DermaMNIST(split="train", transform= transfomrms,download=True,size=224)
val_dataset = DermaMNIST(split="val",transform=transfomrms, download=True,size=224)

loader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
loader_val = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
# Definir el tamaño del subconjunto que deseas usar (por ejemplo, 10%)
subset_size = int(0.1 * len(train_dataset))

# Crear subconjuntos con índices aleatorios
train_indices = torch.randperm(len(train_dataset))[:subset_size]
val_indices = torch.randperm(len(val_dataset))[:subset_size*2]
test_indices = torch.randperm(len(test_dataset))[:subset_size*2]

# Crear los subconjuntos utilizando los índices seleccionados
train_subset = Subset(train_dataset, train_indices)
val_subset = Subset(val_dataset, val_indices)
test_subset = Subset(test_dataset, test_indices)

Using downloaded and verified file: C:\Users\34658\.medmnist\dermamnist_224.npz
Using downloaded and verified file: C:\Users\34658\.medmnist\dermamnist_224.npz
Using downloaded and verified file: C:\Users\34658\.medmnist\dermamnist_224.npz


In [4]:
# start a new wandb run to track this script
import wandb
wandb.finish()
wandb.init(
    # set the wandb project where this run will be logged
    project="Test",
    name="MyComputer - Tests",

    # track hyperparameters and run metadata
    config={
        "N_images" : 1,
        "learning_rate": 0.001,
        "dataset": "Expert 1",
        "epochs": 50,
    }
)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chermar (chermar-universitat-polit-cnica-de-val-ncia). Use `wandb login --relogin` to force relogin


In [5]:
import os
import sys
sys.path.append("..")


loader_train = DataLoader(train_subset, batch_size=32, shuffle=True)
loader_val = DataLoader(val_dataset, batch_size=32, shuffle=True)
loader_test = DataLoader(test_subset, batch_size=32, shuffle=True)

In [64]:
import lightning.pytorch as pl
import torch
import torch.nn as nn
import torchmetrics as tm
from torchmetrics.classification import MulticlassConfusionMatrix
import matplotlib.pyplot as plt

class Classification(pl.LightningModule):
    def __init__(self, model, device):
        super().__init__()
        self.save_hyperparameters(ignore=("model",))

        self.model = model

        self.loss_fn = nn.CrossEntropyLoss()

        self.confusion_matrix = MulticlassConfusionMatrix(num_classes=self.model.classes).to(device)
        self.auc_metric = tm.AUROC(num_classes=self.model.classes, task="multiclass").to(device)  # Definir métrica AUROC para clasificación multiclase

    def forward(self, x):
        return self.model(x)

    def training_step(self, x, y):
        y_hat = self.model(x)
        y_hot = self.transform_classes(y)
        loss = self.loss_fn(y_hat, y_hot.squeeze())
        # Obtener la clase predicha
        y_pred = torch.argmax(y_hat, dim=1)
        # Calcular métricas
        loss.backward()
        self.confusion_matrix.update(y_pred, y.squeeze())
        self.auc_metric(y_hat, y_hot)

        precision, recall, f1_score, ACC, AUC = self.calculate_metrics_from_confusion_matrix()

        return {"loss": loss, "ACC": ACC, "recall": recall, "precision": precision, "f1_score": f1_score, "AUC": AUC}

    def validation_step(self, x, y):
        y_hat = self.model(x)
        y = self.transform_classes(y)
        loss = self.loss_fn(y_hat, y.squeeze())
        # Obtener la clase predicha
        y_pred = torch.argmax(y_hat, dim=1)

        # Calcular métricas
        self.confusion_matrix.update(y_pred, y)

        precision, recall, f1_score, ACC, AUC = self.calculate_metrics_from_confusion_matrix()

        return {"loss": loss, "ACC": ACC, "precision" : precision, "recall": recall, "f1_score" : f1_score, "AUC": AUC}
    def calculate_metrics_from_confusion_matrix(self):
      confusion_matrix = self.confusion_matrix.compute()
      # Verdaderos positivos por clase (diagonal de la matriz)
      true_positives = torch.diag(confusion_matrix)

      # Predicciones totales por clase (sumar columnas)
      predicted_positives = confusion_matrix.sum(dim=0)

      # Ejemplos reales por clase (sumar filas)
      actual_positives = confusion_matrix.sum(dim=1)

      # Calcular Precision, Recall, F1 por clase
      precision = (true_positives / (predicted_positives + 1e-8)).mean()  # Añadir pequeña constante para evitar división por 0
      recall = (true_positives / (actual_positives + 1e-8)).mean()
      f1 = 2 * (precision * recall) / (precision + recall + 1e-8)

      # Calcular ACC
      ACC = true_positives.sum() / confusion_matrix.sum()

      # Calcular el AUC
      AUC = self.auc_metric.compute()
      # Retornar las métricas
      return precision, recall, f1, ACC, AUC
    def transform_classes(self, y):
        # Convertir las clases a un formato de one-hot encoding
        return torch.nn.functional.one_hot(y.to(torch.int64), num_classes=self.model.classes).to(float)
    def restart_epoch(self, plot = False):
        if plot:
            self.confusion_matrix.plot()
            plt.show()
        self.confusion_matrix.reset()
        self.auc_metric.reset()
    def configure_optimizers(self, learning_rate=0.001, betas=(0.9, 0.999), factor=0.1, patience=5):
        optimizer = torch.optim.Adam(self.model.parameters(),
                                     lr=learning_rate,
                                     betas=betas)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                               factor=factor,
                                                               patience=patience)
        return optimizer, scheduler

In [65]:
import torchmetrics as tm
from torch import nn

batch = next(iter(loader_train))
x , y = batch
device  = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model =  ResNet(img_channels=3, num_layers=18, block=BasicBlock, num_classes=7)
trainer = Classification(model, device)

trainer.training_step(x, y)




RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [48]:
#train a model
model =  ResNet(img_channels=3, num_layers=18, block=BasicBlock, num_classes=7)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
trainer = Classification(model, device)



train_model(model, loader_train, loader_val, trainer,  device, num_epochs=2, classification = True)
test_model(model, loader_test, trainer, device, classification = True)

tensor([0., 0., 0., 0., 0., 1., 0.], dtype=torch.float64) tensor([ 0.0071, -0.4821,  0.0485,  0.3912, -0.5942,  0.2623,  0.1378],
       grad_fn=<SelectBackward0>)


ValueError: Either `preds` and `target` both should have the (same) shape (N, ...), or `target` should be (N, ...) and `preds` should be (N, C, ...).